In [ ]:
import camelot
import pandas as pd
import matplotlib


In [ ]:
# Check if dependencies are installed correctly
import tkinter
import ghostscript

import ctypes
from ctypes.util import find_library
import os 
os.environ['PATH'] += os.pathsep + r'C:\Program Files\gs\gs10.03.1\bin'
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

## Getting started

### Understand configurations and package

#### Processing background lines

In [ ]:
tables = camelot.read_pdf('../data/background_lines.pdf', process_background=True)
tables[1].df

#### Visual debugging

In [ ]:
tables = camelot.read_pdf('../data/foo.pdf')
tables
camelot.plot(tables[0], kind='text').show()

In [ ]:
camelot.plot(tables[0], kind='grid').show()

In [ ]:
camelot.plot(tables[0], kind='contour').show()

#### Specifiy table areas

In [ ]:
tables = camelot.read_pdf('../data/table_areas.pdf', flavor='stream', table_areas=['316,499,566,337'])
tables[0].df

### Strip character from text

In [ ]:
tables = camelot.read_pdf('../data/table_extraction_strip.pdf', flavor='stream', strip_text=' .\n')
tables[0].df

### Specify column seperators

In [ ]:
tables = camelot.read_pdf('../data/column_separators.pdf', flavor='stream', columns=['72,95,209,327,442,529,566,606,683'])
tables[0].df